In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('/opt/data/california-worldcover-chips-osm-multilabels.parquet')
df.shape

(83609, 8)

In [3]:
df.head()

,onehot_count,onehot_area,onehot_length,string_labels,col,row,geometry,split
chip_id,,,,,,,,
193c3415fc237,"[24, 24, 1344, 388, 472, 21, 453, 2, 8, 0, 0, ...","[58033.967200952815, 58033.967200952815, 23937...","[4913.205774298301, 4913.205774298301, 74943.4...","[amenity=*, amenity=parking, building=*, build...",78080,96000,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
1639b12583426,"[148, 148, 4171, 1245, 1891, 52, 938, 13, 32, ...","[160073.78667008967, 160073.78667008967, 80191...","[20966.344659970102, 20966.344659970102, 24881...","[amenity=*, amenity=parking, building=*, build...",78080,95744,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
253904d963d6a,"[65, 65, 200, 104, 76, 0, 18, 2, 0, 0, 0, 0, 0...","[101980.95118259436, 101980.95118259436, 71246...","[10597.989172272657, 10597.989172272657, 16664...","[amenity=*, amenity=parking, building=*, build...",77824,95744,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
00530dca88d4a,"[9, 9, 4897, 688, 4026, 0, 172, 10, 1, 0, 0, 0...","[17637.908172952746, 17637.908172952746, 10930...","[1925.4953839123748, 1925.4953839123748, 32065...","[amenity=*, amenity=parking, building=*, build...",77824,95488,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
0e554bbdbe661,"[80, 80, 4358, 1262, 2386, 59, 597, 27, 27, 0,...","[131950.60085068244, 131950.60085068244, 98472...","[13536.03447682216, 13536.03447682216, 283601....","[amenity=*, amenity=parking, building=*, build...",78080,95488,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train


In [4]:
OSM_counts = np.stack(df.onehot_count)
print(OSM_counts.shape)
print(OSM_counts)

(83609, 99)
[[  24   24 1344 ...   53    0    0]
 [ 148  148 4171 ...  166    0    1]
 [  65   65  200 ...   29    0    1]
 ...
 [   4    4    1 ...   77    0   40]
 [   0    0    0 ...    4    0   12]
 [   0    0    0 ...    0    0    7]]


In [5]:
def autocomplete_query(
    constraints: tuple[int],  # user contraints, ex: (0,0,...,1,-1)
    df: pd.DataFrame,
    n_samples: int = 1,
):
    df = df.query("split == 'train'")
    counts = np.stack(df.onehot_count)
    kept_rows_mask = np.array((True,) * df.shape[0])  # mask of kept rows
    constrained_cols_ix = np.nonzero(constraints)[0]  # indices of constrained columns
    
    for col_ix in constrained_cols_ix:
        constraint = constraints[col_ix]
        if constraint == 1:
            kept_rows_mask *= (counts[:, col_ix] > 0)
        elif constraint == -1:
            kept_rows_mask *= (counts[:, col_ix] == 0)

    print(kept_rows_mask.sum())
    if kept_rows_mask.sum() > 0:
        df = df.iloc[kept_rows_mask]  # keep rows that satisfy the constraints
        return df.sample(n=n_samples, replace=True, axis=0, random_state=42)  # sample w replacement
    else:
        return df[0:0]  # return 0 rows

In [6]:
example_constraints = np.array((0,) * 99)
example_constraints[0] = 1
example_constraints[1] = 1
example_constraints[2] = -1
example_constraints[3] = -1
example_constraints[10] = 1
example_constraints

array([ 1,  1, -1, -1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [7]:
# %%debug
autocomplete_query(constraints=example_constraints, df=df, n_samples=20)

12


,onehot_count,onehot_area,onehot_length,string_labels,col,row,geometry,split
chip_id,,,,,,,,
29a5d117b899a,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 31, 2, 29, 0, 49, ...","[1387.1771053231764, 1387.1771053231764, 0.0, ...","[175.3093662846612, 175.3093662846612, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",59648,62208,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
055c53964deb1,"[5, 5, 0, 0, 0, 0, 0, 0, 0, 9, 9, 0, 0, 12, 0,...","[1588.7106419486881, 1588.7106419486881, 0.0, ...","[372.32445834181027, 372.32445834181027, 0.0, ...","[amenity=*, amenity=parking, crop=*, crop=fiel...",55296,62464,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
10c312357b680,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 12, 3, 9, 0, 11, 5...","[1936.0009179324309, 1936.0009179324309, 0.0, ...","[189.24664394866443, 189.24664394866443, 0.0, ...","[amenity=*, amenity=parking, crop=*, crop=fiel...",63744,66816,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
1e164cc44d2c8,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 6, 2, 3, 1, 102, 0...","[1480.8195388180136, 1480.8195388180136, 0.0, ...","[186.3072349785997, 186.3072349785997, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",66560,64512,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
35bfad9302303,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 9, 9, 0, 0, 27, 4,...","[5441.174967043069, 5441.174967043069, 0.0, 0....","[295.1525894512135, 295.1525894512135, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",55296,62976,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
29a5d117b899a,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 31, 2, 29, 0, 49, ...","[1387.1771053231764, 1387.1771053231764, 0.0, ...","[175.3093662846612, 175.3093662846612, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",59648,62208,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
065dc6317a55e,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 7, 5, 2, 0, 51, 3,...","[705.921544141705, 705.921544141705, 0.0, 0.0,...","[111.81848307312679, 111.81848307312679, 0.0, ...","[amenity=*, amenity=parking, crop=*, crop=fiel...",66816,64768,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
1c9b769f462fe,"[2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 4, 2, ...","[2136.42317929564, 2136.42317929564, 0.0, 0.0,...","[318.6417921581011, 318.6417921581011, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",50432,61440,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
29a5d117b899a,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 31, 2, 29, 0, 49, ...","[1387.1771053231764, 1387.1771053231764, 0.0, ...","[175.3093662846612, 175.3093662846612, 0.0, 0....","[amenity=*, amenity=parking, crop=*, crop=fiel...",59648,62208,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
